In [ ]:
import polars as pl # análogo ao pandas - para manipulação de dados
import requests #para acessar a API

In [ ]:
#chaves de acessos sites
TOMORROW_API_KEY = "kd89fhSM3OZQO2WByYaZwYprtuHEqMUI"
EAI_API_KEY      = "ZY9uLnyuz9SpVQ3S7b8FKhKrOWfRamu5abaISqug"

# EIA - Consumo de energia horária

In [ ]:
EAI_API_URL = "https://api.eia.gov/v2/electricity/rto/region-data/data/"

eia_params = {
    "api_key": EAI_API_KEY,
    "frequency": "hourly",
    "data[0]" : "value",
    "facets[type][]": ["D", "DF"],
    "facets[respondent][]" : "LDWP",
    "start": "2025-02-03T00",
    "end": "2025-02-04T00",
    "offset": 0,
    "length": 5000
}

In [ ]:
response = requests.get(EAI_API_URL, params=eia_params)

In [ ]:
# converter para json o objeto response
response_json = response.json()

# pega os dados
data = response_json["response"]["data"]

In [ ]:
# cria o dataframe a partir dos dados
eia_df = pl.DataFrame(data)

In [ ]:
eia_df.head()

period,respondent,respondent-name,type,type-name,value,value-units
str,str,str,str,str,str,str
"""2025-02-04T00""","""LDWP""","""Los Angeles Department of Wate…","""D""","""Demand""","""2584""","""megawatthours"""
"""2025-02-04T00""","""LDWP""","""Los Angeles Department of Wate…","""DF""","""Day-ahead demand forecast""","""2625""","""megawatthours"""
"""2025-02-03T23""","""LDWP""","""Los Angeles Department of Wate…","""D""","""Demand""","""2288""","""megawatthours"""
"""2025-02-03T23""","""LDWP""","""Los Angeles Department of Wate…","""DF""","""Day-ahead demand forecast""","""2575""","""megawatthours"""
"""2025-02-03T22""","""LDWP""","""Los Angeles Department of Wate…","""D""","""Demand""","""2321""","""megawatthours"""


In [ ]:
# converte os valores de string para int
eia_df = eia_df.with_columns(
    pl.col("value").str.to_integer().alias("value")
)

In [ ]:
# somente dados da demanda
demand_df = eia_df.filter(pl.col("type-name") == "Demand").sort("period", descending=False)

# previsao oficial da demanda
forecast_df = eia_df.filter(pl.col("type-name") == "Day-ahead demand forecast").sort("period", descending=False)

In [ ]:
demand_df.head()

period,respondent,respondent-name,type,type-name,value,value-units
str,str,str,str,str,i64,str
"""2025-02-03T00""","""LDWP""","""Los Angeles Department of Wate…","""D""","""Demand""",2650,"""megawatthours"""
"""2025-02-03T01""","""LDWP""","""Los Angeles Department of Wate…","""D""","""Demand""",2739,"""megawatthours"""
"""2025-02-03T02""","""LDWP""","""Los Angeles Department of Wate…","""D""","""Demand""",2875,"""megawatthours"""
"""2025-02-03T03""","""LDWP""","""Los Angeles Department of Wate…","""D""","""Demand""",2939,"""megawatthours"""
"""2025-02-03T04""","""LDWP""","""Los Angeles Department of Wate…","""D""","""Demand""",2836,"""megawatthours"""


In [ ]:
forecast_df.head()

period,respondent,respondent-name,type,type-name,value,value-units
str,str,str,str,str,i64,str
"""2025-02-03T00""","""LDWP""","""Los Angeles Department of Wate…","""DF""","""Day-ahead demand forecast""",2191,"""megawatthours"""
"""2025-02-03T01""","""LDWP""","""Los Angeles Department of Wate…","""DF""","""Day-ahead demand forecast""",2314,"""megawatthours"""
"""2025-02-03T02""","""LDWP""","""Los Angeles Department of Wate…","""DF""","""Day-ahead demand forecast""",2497,"""megawatthours"""
"""2025-02-03T03""","""LDWP""","""Los Angeles Department of Wate…","""DF""","""Day-ahead demand forecast""",2637,"""megawatthours"""
"""2025-02-03T04""","""LDWP""","""Los Angeles Department of Wate…","""DF""","""Day-ahead demand forecast""",2653,"""megawatthours"""
